In [189]:
import pandas as pd
from bs4 import BeautifulSoup

import requests
import textacy
import os
from thefuzz import process
os.chdir('../..')
en_md = spacy.load("en_core_web_md")

In [13]:
FETCH_URL_TEMPLATE = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:{}&from=EN"
def fetch_eurlex(celex: str):
    celex = celex.replace("/", "%2F")
    url = FETCH_URL_TEMPLATE.format(celex)
    r = requests.get(
        url
    )
    if r.status_code == 200:
        return r.content


In [14]:
soup = BeautifulSoup(fetch_eurlex("52021PC0206"))
text = soup.getText()

In [25]:
t= text.split("\n")

a = [w for w in t if w!='']

lines = pd.Series(a)

In [33]:
lines[lines.str.contains("TITLE I")]

5       REGULATION OF THE EUROPEAN PARLIAMENT AND OF T...
188     REGULATION OF THE EUROPEAN PARLIAMENT AND OF T...
1892       See Proposal for a REGULATION OF THE EUROPE...
2219    (a)Proposal for a REGULATION OF THE EUROPEAN P...
2221    (a)Amended proposal for a REGULATION OF THE EU...
dtype: object

In [146]:
l2['TITLE']

KeyError: 'TITLE'

In [44]:
l2 = lines[346:]

In [51]:
l3 = l2[l2.str.contains(r"^\(\d+\)")]

In [63]:
l4 = l3[l3.str.len()>4]

In [75]:
l5 = l4.str.replace(r"\(\d+\)", "", regex=True)

In [104]:
# p = l5.apply(phrasemachine.get_phrases)
# p.apply(lambda x: x['counts'])

In [106]:
import spacy
en = spacy.load('en_core_web_lg')

In [178]:
text  = l5.loc[405]


text

‘post-market monitoring’ means all activities carried out by providers of AI systems to proactively collect and review experience gained from the use of AI systems they place on the market or put into service for the purpose of identifying any need to immediately apply any necessary corrective or preventive actions;

In [187]:
list(en_lg(text).noun_chunks)

[all activities,
 providers,
 AI systems,
 the use,
 AI systems,
 they,
 the market,
 service,
 the purpose,
 any need,
 any necessary corrective or preventive actions]

In [190]:
list(en_md(text).noun_chunks)

[post-market monitoring,
 all activities,
 providers,
 AI systems,
 experience,
 the use,
 AI systems,
 they,
 the market,
 service,
 the purpose,
 any need,
 any necessary corrective or preventive actions]

In [184]:
list(en_sm(text).noun_chunks)

[post-market monitoring,
 all activities,
 providers,
 AI systems,
 experience,
 the use,
 AI systems,
 they,
 the market,
 service,
 the purpose,
 any need,
 any necessary corrective or preventive actions]

In [113]:
def extract_noun_chunks(doc):
    try:
        return [
            chunk.text
            for chunk in textacy.extract.basics.noun_chunks(doc)
            if not len(chunk) == 1 or chunk[0].pos_ in {"PROPN", "NOUN"}
        ]
    except AttributeError:
        return []

In [193]:
phrases = l5.apply(en_md).apply(extract_noun_chunks)

In [195]:
phrases = phrases.apply(lambda a: [x.strip("‘") for x in a])
pcount = phrases.explode().value_counts()
pcount

In [204]:
phrases = list(pcount.index)
phrases = [a.lower() for a in phrases]

In [207]:
chunk_memes = pd.read_parquet('data/s2orc/chunk_meme_mappings/reduced_300_big_cleaned_phrase-bert_eps_0.2_min_clust_size_3.parquet')

In [212]:
a = chunk_memes[chunk_memes['chunk'].isin(phrases)]

In [230]:
a.to_csv('chunks_from_ai_act_phrasebert.csv', index=True)

In [226]:
phrase = phrases[0]

In [229]:
process.extractOne(phrase,chunk_memes['chunk'])

('ai system', 100, 2630395)